In [ ]:

import re
import sys
import os
import random
import string
import time
import pickle
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import optimizers
from tensorflow.keras.layers import add
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.utils import plot_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Dense, LSTM, GRU, TimeDistributed
from tensorflow.keras.layers import Embedding, Dropout, Bidirectional, Concatenate, Lambda
import glob
import pandas as pd
np.random.seed(1)


In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

os.chdir("/content/drive/Othercomputers/Dell")

Mounted at /content/drive


In [ ]:
os.chdir("/content/drive/Othercomputers/Dell")
!pwd

/content/drive/Othercomputers/Dell


In [ ]:
path = !pwd
path
#path = /content/drive/Othercomputers/Dell/Action-Segmentation-Project

['/content/drive/Othercomputers/Dell']

In [ ]:
#get all the optofoce_data.csv

# files = ['/media/sf_Action-Segmentation-Project/avocado','']
# for file in files:
#     for name in glob.glob(f"{file}/clutter/[0-9]*/optoforce_data.csv"):
#         print(name)

# files = glob.glob(f"/media/sf_Action-Segmentation-Project/avocado/clutter/[0-9]*/optoforce_data.csv")
files = glob.glob(f"/content/drive/Othercomputers/Dell/Action-Segmentation-Project/avocado/clutter/[0-9]*/optoforce_data.csv")
files

['/content/drive/Othercomputers/Dell/Action-Segmentation-Project/avocado/clutter/7/optoforce_data.csv',
 '/content/drive/Othercomputers/Dell/Action-Segmentation-Project/avocado/clutter/4/optoforce_data.csv',
 '/content/drive/Othercomputers/Dell/Action-Segmentation-Project/avocado/clutter/5/optoforce_data.csv',
 '/content/drive/Othercomputers/Dell/Action-Segmentation-Project/avocado/clutter/9/optoforce_data.csv',
 '/content/drive/Othercomputers/Dell/Action-Segmentation-Project/avocado/clutter/8/optoforce_data.csv',
 '/content/drive/Othercomputers/Dell/Action-Segmentation-Project/avocado/clutter/2/optoforce_data.csv',
 '/content/drive/Othercomputers/Dell/Action-Segmentation-Project/avocado/clutter/3/optoforce_data.csv',
 '/content/drive/Othercomputers/Dell/Action-Segmentation-Project/avocado/clutter/10/optoforce_data.csv',
 '/content/drive/Othercomputers/Dell/Action-Segmentation-Project/avocado/clutter/6/optoforce_data.csv',
 '/content/drive/Othercomputers/Dell/Action-Segmentation-Projec

In [ ]:
#labels = glob.glob(f"/media/sf_Action-Segmentation-Project/avocado/clutter/[0-9]*/labels")
labels = glob.glob(f"/content/drive/Othercomputers/Dell/Action-Segmentation-Project/avocado/clutter/[0-9]*/labels")

labels

['/content/drive/Othercomputers/Dell/Action-Segmentation-Project/avocado/clutter/7/labels',
 '/content/drive/Othercomputers/Dell/Action-Segmentation-Project/avocado/clutter/4/labels',
 '/content/drive/Othercomputers/Dell/Action-Segmentation-Project/avocado/clutter/5/labels',
 '/content/drive/Othercomputers/Dell/Action-Segmentation-Project/avocado/clutter/9/labels',
 '/content/drive/Othercomputers/Dell/Action-Segmentation-Project/avocado/clutter/8/labels',
 '/content/drive/Othercomputers/Dell/Action-Segmentation-Project/avocado/clutter/2/labels',
 '/content/drive/Othercomputers/Dell/Action-Segmentation-Project/avocado/clutter/3/labels',
 '/content/drive/Othercomputers/Dell/Action-Segmentation-Project/avocado/clutter/10/labels',
 '/content/drive/Othercomputers/Dell/Action-Segmentation-Project/avocado/clutter/6/labels',
 '/content/drive/Othercomputers/Dell/Action-Segmentation-Project/avocado/clutter/1/labels']

In [ ]:
frames = []
data = []
for f in files:
    # print(f)
    # print(l)
    data_df = pd.read_csv(f)
    #data.append(data_df)
    data_df["label"] = ""
    frames.append(data_df)


#data = pd.concat(frames, ignore_index=True)


In [ ]:
len(frames)

10

In [ ]:
frames[0]

,time,index_x,index_y,index_z,middle_x,middle_y,middle_z,ring_x,ring_y,ring_z,thumb_x,thumb_y,thumb_z,label
0,1635185477459841536,-0.16,-0.16,0.64,-0.16,-0.40,0.08,-0.08,-0.16,0.32,0.32,0.00,1.12,
1,1635185477460862720,-0.16,-0.16,0.64,-0.24,-0.40,0.08,-0.16,-0.16,0.32,0.32,0.00,1.20,
2,1635185477461578496,-0.16,-0.16,0.64,-0.08,-0.32,0.08,-0.16,-0.08,0.32,0.24,-0.08,1.20,
3,1635185477462545664,-0.08,-0.16,0.64,-0.08,-0.24,0.00,-0.16,-0.08,0.32,0.24,0.00,1.20,
4,1635185477463597312,-0.08,-0.08,0.64,-0.08,-0.24,0.00,-0.16,0.00,0.32,0.24,0.00,1.20,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72505,1635185564093448960,0.08,-0.56,0.08,0.40,-0.16,-0.08,0.16,0.00,-0.40,0.56,-0.08,0.40,
72506,1635185564094279424,0.08,-0.64,0.08,0.40,-0.24,-0.08,0.16,0.00,-0.40,0.48,-0.16,0.40,
72507,1635185564095298560,0.00,-0.64,0.00,0.32,-0.24,-0.08,0.00,-0.08,-0.32,0.48,-0.16,0.40,
72508,1635185564096268032,-0.08,-0.64,0.00,0.32,-0.24,-0.08,0.00,-0.08,-0.40,0.40,-0.24,0.40,


In [ ]:
action_segment_td = []
ground_truth_actions = []

for l in labels:
    td_per_file = []
    gt_actions_per_file = []
    with open (l) as f:
        for line in f:
            x, y  = line.split(';')
            #start_time, end_time = x.split(':')
            td_per_file.append(x)
            gt_actions_per_file.append(y.strip('\n'))
        action_segment_td.append(td_per_file)
        ground_truth_actions.append(gt_actions_per_file)


In [ ]:
action_segment_td #timestmps

[['1635185484881988107:1635185508960098923',
  '1635185508960098923:1635185540489498512',
  '1635185540489498512:1635185544057936064',
  '1635185544057936064:1635185549350041664',
  '1635185549350041664:1635185554816474229',
  '1635185554816474229:1635185560655390372'],
 ['1634917079582315741:1634917108946914859',
  '1634917108946914859:1634917129420464685',
  '1634917129420464685:1634917131539426547',
  '1634917131539426547:1634917134957114204',
  '1634917134957114204:1634917140855676126',
  '1634917140855676126:1634917153391803434'],
 ['1634918557303008582:1634918568217991596',
  '1634918568217991596:1634918593817386818',
  '1634918593817386818:1634918646595079287',
  '1634918646595079287:1634918651382668779',
  '1634918651382668779:1634918657623002640',
  '1634918657623002640:1634918664418463962'],
 ['1635186448159756509:1635186459390779330',
  '1635186459390779330:1635186464698664958',
  '1635186464698664958:1635186467998475334',
  '1635186467998475334:1635186475265396949',
  '1635

In [ ]:
len(ground_truth_actions) #labels


10

In [ ]:
ground_truth_actions


[['move-in', 'manipulate', 'grasp', 'pick-up', 'move-out', 'drop'],
 ['move-in', 'manipulate', 'grasp', 'pick-up', 'move-out', 'drop'],
 ['move-in', 'manipulate', 'grasp', 'pick-up', 'move-out', 'drop'],
 ['move-in', 'manipulate', 'move-in', 'grasp', 'pick-up', 'move-out', 'drop'],
 ['move-in',
  'manipulate',
  'move-out',
  'move-in',
  'manipulate',
  'grasp',
  'pick-up',
  'move-out',
  'drop'],
 ['move-in', 'manipulate', 'grasp', 'pick-up', 'move-out', 'drop'],
 ['move-in',
  'manipulate',
  'move-out',
  'move-in',
  'manipulate',
  'move-out',
  'move-in',
  'grasp',
  'pick-up',
  'move-out',
  'drop'],
 ['move-in',
  'manipulate',
  'move-out',
  'move-in',
  'grasp',
  'pick-up',
  'move-out',
  'drop'],
 ['move-in',
  'manipulate',
  'move-out',
  'move-in',
  'manipulate',
  'move-out',
  'move-in',
  'grasp',
  'pick-up',
  'move-out',
  'drop'],
 ['move-in',
  'manipulate',
  'move-out',
  'move-in',
  'manipulate',
  'grasp',
  'pick-up',
  'move-out',
  'drop']]

In [ ]:
for df, duration_of_actions, labels in zip(frames,action_segment_td, ground_truth_actions):
    condition = []
    for actions in duration_of_actions:

        #print(actions)
        start_time, end_time = actions.split(':')
        #print(start_time)
        #print(end_time)
        #break
        condition.append(df['time'].between(int(start_time),int(end_time)))

    df['label'] = np.select(condition,labels,default=None)
    df.dropna(inplace=True)
#samples = pd.concat(frames,ignore_index=True)

In [ ]:
len(frames[0])


62956

In [ ]:
unique_actions = set()

for frame in frames:
    for label in frame['label']:
        unique_actions.add(label)


In [ ]:
unique_actions


{'drop', 'grasp', 'manipulate', 'move-in', 'move-out', 'pick-up'}

In [ ]:
one_hot_encoding_acts = pd.get_dummies(list(unique_actions))
one_hot_encoding_acts


,drop,grasp,manipulate,move-in,move-out,pick-up
0,0,1,0,0,0,0
1,0,0,0,1,0,0
2,1,0,0,0,0,0
3,0,0,1,0,0,0
4,0,0,0,0,0,1
5,0,0,0,0,1,0


In [ ]:
actions_per_seq = []
for frame in frames:
    action_encodings = []
    for i in range(0, len(frame)):
        action_encodings.append(one_hot_encoding_acts[frame['label'].iloc[i]])
    actions_per_seq.append(action_encodings)

In [ ]:
len(actions_per_seq)


10

In [ ]:
len(actions_per_seq[0])


62956

In [ ]:
# data_array = []
# for frame in frames:
#     cols = list(frame.columns[:-1])
#     data_array.append(cols)
#     matric


In [ ]:
frames[0].columns

Index(['time', 'index_x', 'index_y', 'index_z', 'middle_x', 'middle_y',
       'middle_z', 'ring_x', 'ring_y', 'ring_z', 'thumb_x', 'thumb_y',
       'thumb_z', 'label'],
      dtype='object')

In [ ]:
features = ['index_x', 'index_y', 'index_z','middle_x','middle_y','middle_z','ring_x','ring_y','ring_z','thumb_x', 'thumb_y', 'thumb_z' ]
numeric_features = frames[0][features]
#numeric_features.head()
np.array(numeric_features)

array([[ 0.16, -0.24,  0.48, ...,  0.4 ,  0.08,  1.2 ],
       [ 0.  , -0.24,  0.48, ...,  0.4 ,  0.  ,  1.2 ],
       [ 0.  , -0.16,  0.48, ...,  0.24,  0.  ,  1.2 ],
       ...,
       [ 0.  , -0.56,  0.16, ...,  0.8 , -0.56,  0.64],
       [ 0.08, -0.56,  0.24, ...,  0.8 , -0.56,  0.64],
       [ 0.16, -0.48,  0.24, ...,  0.8 , -0.4 ,  0.64]])

In [ ]:
numeric_features.head()


,index_x,index_y,index_z,middle_x,middle_y,middle_z,ring_x,ring_y,ring_z,thumb_x,thumb_y,thumb_z
6462,0.16,-0.24,0.48,0.16,-0.24,0.08,0.08,0.00,-0.16,0.40,0.08,1.20
6463,0.00,-0.24,0.48,0.08,-0.24,0.00,0.00,0.00,-0.16,0.40,0.00,1.20
6464,0.00,-0.16,0.48,0.00,-0.32,0.00,-0.16,-0.08,-0.16,0.24,0.00,1.20
6465,0.00,-0.16,0.48,-0.08,-0.32,0.00,-0.16,-0.16,-0.16,0.24,-0.08,1.12
6466,0.00,-0.24,0.40,-0.08,-0.32,0.08,-0.16,-0.16,-0.16,0.24,-0.08,1.12


In [ ]:
numeric_features_per_seq = [np.array(frames[i][features]) for i in range(len(frames))]

In [ ]:
features_per_seq = np.array(numeric_features_per_seq)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


In [ ]:
features_per_seq[0]


array([[ 0.16, -0.24,  0.48, ...,  0.4 ,  0.08,  1.2 ],
       [ 0.  , -0.24,  0.48, ...,  0.4 ,  0.  ,  1.2 ],
       [ 0.  , -0.16,  0.48, ...,  0.24,  0.  ,  1.2 ],
       ...,
       [ 0.  , -0.56,  0.16, ...,  0.8 , -0.56,  0.64],
       [ 0.08, -0.56,  0.24, ...,  0.8 , -0.56,  0.64],
       [ 0.16, -0.48,  0.24, ...,  0.8 , -0.4 ,  0.64]])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test  = train_test_split(features_per_seq,np.array(actions_per_seq),shuffle=False, stratify=None)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


In [ ]:
len(X_train[0])


62956

In [ ]:
X_input = X_train[0].reshape(1,62956,12)
y_input = np.array(actions_per_seq[0])
y_input = y_input.reshape(1,62956,6)

In [ ]:
def repeat_vector(args):
    layer_to_repeat = args[0]
    sequence_layer = args[1]
    return RepeatVector(K.shape(sequence_layer)[1])(layer_to_repeat)

In [ ]:
N_TIMESTEPS = len(X_train[0])
N_FEATURES = 12

In [ ]:
from keras.layers import RepeatVector

input_ = Input(shape=(N_TIMESTEPS,N_FEATURES),dtype='float64')
lstm1 = LSTM(units=100, name = 'lstm1')(input_)
#rv = Lambda(repeat_vector, output_shape=(None,12))([input_,lstm1])
rv = RepeatVector(N_TIMESTEPS)(lstm1)
lstm2 = LSTM(units = 100, return_sequences=True, name = 'lstm2')(rv)
td = TimeDistributed(Dense(units = 6, activation='softmax'))(lstm2)


model_1 = Model(inputs = input_, outputs = td)
model_1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model_1.summary()

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, 62956, 12)]       0         
                                                                 
 lstm1 (LSTM)                (None, 100)               45200     
                                                                 
 repeat_vector_11 (RepeatVec  (None, 62956, 100)       0         
 tor)                                                            
                                                                 
 lstm2 (LSTM)                (None, 62956, 100)        80400     
                                                                 
 time_distributed_11 (TimeDi  (None, 62956, 6)         606       
 stributed)                                                      
                                                                 
Total params: 126,206
Trainable params: 126,206
Non-traina

In [ ]:
X_train[0].shape

(62956, 12)

In [ ]:
model_history = model_1.fit(X_input,y_input, epochs=200,
                          verbose = 1)

Epoch 1/200
1/1 [==============================] - 8s 8s/step - loss: 1.7692 - accuracy: 0.3163
Epoch 2/200
1/1 [==============================] - 5s 5s/step - loss: 1.6847 - accuracy: 0.4145
Epoch 3/200
1/1 [==============================] - 5s 5s/step - loss: 1.6054 - accuracy: 0.4145
Epoch 4/200
1/1 [==============================] - 5s 5s/step - loss: 1.5301 - accuracy: 0.4145
Epoch 5/200
1/1 [==============================] - 5s 5s/step - loss: 1.4757 - accuracy: 0.4145
Epoch 6/200
1/1 [==============================] - 5s 5s/step - loss: 1.4691 - accuracy: 0.4145
Epoch 7/200
1/1 [==============================] - 5s 5s/step - loss: 1.4755 - accuracy: 0.4145
Epoch 8/200
1/1 [==============================] - 5s 5s/step - loss: 1.4782 - accuracy: 0.4145
Epoch 9/200
1/1 [==============================] - 5s 5s/step - loss: 1.4777 - accuracy: 0.3161
Epoch 10/200
1/1 [==============================] - 5s 5s/step - loss: 1.4685 - accuracy: 0.3162
Epoch 11/200
1/1 [=====================

KeyboardInterrupt: ignored

In [ ]:
X = list()
Y = list()
X1 = [x1 for x1 in range(5, 301, 5)]
X2 = [x2 for x2 in range(20, 316, 5)]
Y = [y for y in range(35, 331, 5)]

X = np.column_stack((X1, X2))

In [ ]:
X = np.array(X).reshape(20, 3, 2)
Y = np.array(Y).reshape(20, 3, 1)

In [ ]:
X

array([[[  5,  20],
        [ 10,  25],
        [ 15,  30]],

       [[ 20,  35],
        [ 25,  40],
        [ 30,  45]],

       [[ 35,  50],
        [ 40,  55],
        [ 45,  60]],

       [[ 50,  65],
        [ 55,  70],
        [ 60,  75]],

       [[ 65,  80],
        [ 70,  85],
        [ 75,  90]],

       [[ 80,  95],
        [ 85, 100],
        [ 90, 105]],

       [[ 95, 110],
        [100, 115],
        [105, 120]],

       [[110, 125],
        [115, 130],
        [120, 135]],

       [[125, 140],
        [130, 145],
        [135, 150]],

       [[140, 155],
        [145, 160],
        [150, 165]],

       [[155, 170],
        [160, 175],
        [165, 180]],

       [[170, 185],
        [175, 190],
        [180, 195]],

       [[185, 200],
        [190, 205],
        [195, 210]],

       [[200, 215],
        [205, 220],
        [210, 225]],

       [[215, 230],
        [220, 235],
        [225, 240]],

       [[230, 245],
        [235, 250],
        [240, 255]],

       [